In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import make_scorer, confusion_matrix, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV

In [38]:
# Import data
raw_train_df = pd.read_csv("train.csv")
raw_train_label_df = pd.read_csv('train_label.csv')

In [39]:
# feature transform
monthMap = {'January':'1', 'February':'2', 'March':'3',\
            'April':'4', 'May':'5', 'June':'6',\
            'July':'7', 'August':'8', 'September':'9',\
            'October':'10', 'November':'11', 'December':'12'}
def getArrivalDate(row):
    return pd.to_datetime(str(row.arrival_date_year) + '-' + monthMap[row.arrival_date_month] + '-' + str(row.arrival_date_day_of_month))

raw_train_df['arrival_date'] = raw_train_df.apply(getArrivalDate, axis=1)
raw_train_df['stays'] = raw_train_df.apply(lambda row: row.stays_in_weekend_nights + row.stays_in_week_nights, axis=1)
raw_train_df['expected_cost'] = raw_train_df.apply(lambda row: row.adr * row.stays, axis=1)

In [40]:
# Analyze invalid data
invalid_data = raw_train_df[raw_train_df.adr < 0]
invalid_data['arrival_date'] = invalid_data.apply(getArrivalDate, axis=1)
invalid_data['stays'] = invalid_data.apply(lambda row: row.stays_in_weekend_nights + row.stays_in_week_nights, axis=1)
invalid_data['expected_cost'] = invalid_data.apply(lambda row: row.adr * row.stays, axis=1)

print(raw_train_df['arrival_date'].value_counts())
print(invalid_data['arrival_date'].value_counts())
print(invalid_data['is_canceled'].value_counts())

2015-12-05    448
2016-11-07    366
2015-10-16    356
2016-10-13    344
2015-09-18    340
             ... 
2015-12-15     29
2015-11-15     28
2015-12-07     27
2015-11-29     20
2015-12-13     19
Name: arrival_date, Length: 640, dtype: int64
2015-09-17    68
2016-10-06    47
2016-05-12    41
2015-09-02    40
2016-10-20    37
              ..
2015-11-10     1
2017-03-20     1
2016-01-28     1
2016-10-02     1
2016-05-07     1
Name: arrival_date, Length: 490, dtype: int64
0    1229
1     878
Name: is_canceled, dtype: int64


<ipython-input-40-639d520b5cf6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invalid_data['arrival_date'] = invalid_data.apply(getArrivalDate, axis=1)
<ipython-input-40-639d520b5cf6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invalid_data['stays'] = invalid_data.apply(lambda row: row.stays_in_weekend_nights + row.stays_in_week_nights, axis=1)
<ipython-input-40-639d520b5cf6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [41]:
# clean invalid data
raw_train_df.drop(raw_train_df[raw_train_df.adr < 0].index, inplace=True)
raw_train_df.reset_index(drop=True, inplace=True)

In [42]:
raw_train_df.shape

(89424, 36)

In [43]:
# Split validation set and training set
dates = raw_train_df.arrival_date.unique()
train_indices, val_indices = train_test_split(dates, test_size=0.3)
val_df = raw_train_df.set_index('arrival_date').loc[val_indices, :]
train_df = raw_train_df.set_index('arrival_date').loc[train_indices, :]

In [44]:
# Get the target variable from the training set
raw_train_label = raw_train_df['is_canceled']
train_label = train_df['is_canceled']
val_label = val_df['is_canceled']

In [45]:
train_df.reset_index(drop=False, inplace=True)
val_df.reset_index(drop=False, inplace=True)

In [2]:
numericCols = ['lead_time', 'stays', 'stays_in_weekend_nights', 'stays_in_week_nights',\
               'adults', 'children', 'babies', 'previous_cancellations',\
               'previous_bookings_not_canceled', 'booking_changes', 'days_in_waiting_list',\
               'required_car_parking_spaces', 'total_of_special_requests'
              ]
categoryCols = ['hotel', 'arrival_date_year', 'arrival_date_month',\
                'arrival_date_week_number', 'arrival_date_day_of_month', 'meal',\
                'country', 'market_segment', 'distribution_channel',\
                'is_repeated_guest', 'reserved_room_type', 'assigned_room_type',\
                'deposit_type', 'customer_type'
               ]
featureCols = numericCols + categoryCols

In [3]:
train = train_df[featureCols]
val = val_df[featureCols]

NameError: name 'train_df' is not defined

In [77]:
train

,lead_time,stays,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,...,arrival_date_day_of_month,meal,country,market_segment,distribution_channel,is_repeated_guest,reserved_room_type,assigned_room_type,deposit_type,customer_type
0,4,1,0,1,4,0.0,0,0,2,2,...,13,BB,PRT,Complementary,Direct,1,G,G,No Deposit,Transient
1,119,4,2,2,2,0.0,0,0,0,0,...,13,BB,PRT,Online TA,TA/TO,0,E,E,No Deposit,Transient
2,54,1,0,1,2,0.0,0,0,0,0,...,13,BB,USA,Online TA,TA/TO,0,D,D,No Deposit,Transient
3,30,2,0,2,1,2.0,0,0,0,0,...,13,BB,ARG,Online TA,TA/TO,0,G,G,No Deposit,Transient-Party
4,9,2,0,2,2,0.0,0,0,0,0,...,13,BB,PRT,Direct,Direct,0,E,E,No Deposit,Transient
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62737,278,2,0,2,2,0.0,0,1,0,0,...,22,BB,PRT,Groups,TA/TO,0,A,A,Non Refund,Transient-Party
62738,0,1,0,1,2,0.0,0,0,0,0,...,22,BB,ESP,Direct,Direct,0,E,E,No Deposit,Transient
62739,278,2,0,2,2,0.0,0,1,0,0,...,22,BB,PRT,Groups,TA/TO,0,A,A,Non Refund,Transient-Party
62740,278,2,0,2,2,0.0,0,1,0,0,...,22,BB,PRT,Groups,TA/TO,0,A,A,Non Refund,Transient-Party


In [78]:
# ColumnTransformer

# Transforming the categoric columns
cat_si_step = ('si', SimpleImputer(strategy='constant', fill_value='MISSING'))
cat_ohe_step = ('ohe', OneHotEncoder(sparse=True, handle_unknown='ignore'))
cat_steps = [cat_si_step, cat_ohe_step]
cat_pipe = Pipeline(cat_steps)

cat_transformers = [('cat', cat_pipe, categoryCols)]
cat_ct = ColumnTransformer(transformers=cat_transformers)

train_cat_transformed = cat_ct.fit_transform(train_df)

In [79]:
# Transforming the numeric columns
num_si_step = ('si', SimpleImputer(strategy='median'))
num_ss_step = ('ss', StandardScaler())
num_steps = [num_si_step, num_ss_step]
num_pipe = Pipeline(num_steps)

num_transformers = [('num', num_pipe, numericCols)]
num_ct = ColumnTransformer(transformers=num_transformers)

train_num_transformed = num_ct.fit_transform(train_df)

In [80]:
# Retrieving the feature names
cat_pl = cat_ct.named_transformers_['cat']
ohe = cat_pl.named_steps['ohe']
transformed_feature_names = list(ohe.get_feature_names()) + numericCols
print("Total number of features = ", len(transformed_feature_names))

Total number of features =  313


In [81]:
# Combining both categorical and numerical column transformations
ct = ColumnTransformer(transformers=[('cat', cat_pipe, categoryCols), ('num', num_pipe, numericCols)])
train_transformed = ct.fit_transform(train_df)

In [82]:
# feature selection
sel = VarianceThreshold(threshold=(.9 * (1 - .9)))
#rfe = RFE(estimator=lr, n_features_to_select=5, step=1)

In [83]:
# SVD
svd = TruncatedSVD(n_components=50) # best 50

In [84]:
# Logistic regression
lr = LogisticRegression(penalty='l2', C=90.0, max_iter=10000)

In [85]:
# Build pipeline
lr_pipe = Pipeline([('transform', ct), ('svd', svd), ('lr', lr)])

In [86]:
# Decide sample weight
sample_weights = train_df.apply(lambda row: pow(row.expected_cost, 1), axis=1)

In [1]:
# Execute pipeline
lr_pipe.fit(train, train_label, lr__sample_weight=sample_weights)
lr_pipe.score(train, train_label)

NameError: name 'lr_pipe' is not defined

In [57]:
# Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=1126)

# Selecting parameters when Grid Searching
param_grid = {
    'transform__num__si__strategy': ['median'],
    'svd__n_components': [40, 45, 50],
    'lr__penalty': ['l2'],
    'lr__C': [80, 90, 100]
}

gs = GridSearchCV(lr_pipe, param_grid, cv=kf, scoring=make_scorer(log_loss))
gs.fit(train_df, train_label)

print(gs.best_params_)
print(gs.best_score_)

# Getting all the grid search results in a Pandas DataFrame
print(pd.DataFrame(gs.cv_results_))

{'lr__C': 90, 'lr__penalty': 'l2', 'svd__n_components': 40, 'transform__num__si__strategy': 'median'}
6.267368501185082
   mean_fit_time  std_fit_time  mean_score_time  std_score_time param_lr__C  \
0       2.350239      0.129905         0.080649        0.000781          80   
1       2.778026      0.113313         0.081183        0.000854          80   
2       3.228148      0.171731         0.081608        0.000833          80   
3       2.170479      0.178515         0.079649        0.000607          90   
4       2.862789      0.100329         0.081740        0.000662          90   
5       3.024211      0.106975         0.081016        0.001560          90   
6       2.239353      0.116394         0.079338        0.000565         100   
7       2.868658      0.189433         0.082042        0.000894         100   
8       2.952179      0.347093         0.082689        0.001099         100   

  param_lr__penalty param_svd__n_components  \
0                l2                      4

In [ ]:
'''
# Too slow !!!
# SVC
svc = SVC(C=80.0, kernel='rbf', gamma='scale', probability=True, cache_size=200)
svc_pipe = Pipeline([('transform', ct), ('svd', svd), ('svc', svc)])
svc_pipe.fit(train_df, train_label)
svc_pipe.score(train_df, train_label)
'''

In [58]:
# Use hard classification
will_be_canceled = gs.predict(val_df)
val_df['will_be_canceled'] = will_be_canceled

In [59]:
# Use soft classification
will_be_canceled_prob = lr_pipe.predict_proba(val_df)
will_be_canceled = np.empty(shape=will_be_canceled_prob.shape[0])
for i in range(will_be_canceled.shape[0]):
    will_be_canceled[i] = will_be_canceled_prob[i][1]
will_be_canceled = pd.Series(will_be_canceled)

val_df['will_be_canceled'] = will_be_canceled
print(val_df['will_be_canceled'].value_counts())

0.999716    150
0.989749    101
0.988732     99
0.987665     85
0.992460     79
           ... 
0.464647      1
0.818180      1
0.853546      1
0.093077      1
0.211867      1
Name: will_be_canceled, Length: 18918, dtype: int64


/opt/conda/lib/python3.8/site-packages/sklearn/compose/_column_transformer.py:437: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  warnings.warn("Given feature/column names or counts do not match "


In [60]:
def getRevenue(row):
    if row.is_canceled:
        return 0
    return row.expected_cost

def predictRevenue(row):
    return (1.0 - row.will_be_canceled) * row.expected_cost

In [61]:
# Post process
val_df['revenue'] = val_df.apply(getRevenue, axis=1)
val_df['predicted_revenue'] = val_df.apply(predictRevenue, axis=1)

In [62]:
# Aggregate by date
daily_revenue_df = val_df.groupby(['arrival_date']).agg({'revenue':'sum', 'predicted_revenue':'sum'})

In [63]:
thresholds = [0, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000]
ranks = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
daily_revenue_df['calculated_label'] = pd.cut(daily_revenue_df.revenue, bins=thresholds,labels=ranks)
daily_revenue_df['predicted_label'] = pd.cut(daily_revenue_df.predicted_revenue, bins=thresholds,labels=ranks)

In [64]:
daily_revenue_df

,revenue,predicted_revenue,calculated_label,predicted_label
arrival_date,,,,
2015-07-02,16530.645277,18220.816672,1,1
2015-07-04,17488.551606,16128.846680,1,1
2015-07-06,21441.591688,19320.385399,2,1
2015-07-07,10930.818639,12786.020136,1,1
2015-07-08,12327.488839,11945.763111,1,1
...,...,...,...,...
2017-03-02,29771.348973,29219.110969,2,2
2017-03-10,29235.325908,30039.128650,2,3
2017-03-20,32589.526731,29797.047476,3,2


In [67]:
result_df = daily_revenue_df.join(raw_train_label_df.set_index('arrival_date'), how='inner')

In [68]:
result_df['calculated_err'] = result_df.apply(lambda row: abs(row.calculated_label - row.label), axis=1)
result_df['err'] = result_df.apply(lambda row: abs(row.predicted_label - row.label), axis=1)

In [69]:
result_df

,revenue,predicted_revenue,calculated_label,predicted_label,label,calculated_err,err
arrival_date,,,,,,,
2015-07-02,16530.645277,18220.816672,1,1,1.0,0.0,0.0
2015-07-04,17488.551606,16128.846680,1,1,1.0,0.0,0.0
2015-07-06,21441.591688,19320.385399,2,1,2.0,0.0,1.0
2015-07-07,10930.818639,12786.020136,1,1,1.0,0.0,0.0
2015-07-08,12327.488839,11945.763111,1,1,1.0,0.0,0.0
...,...,...,...,...,...,...,...
2017-03-02,29771.348973,29219.110969,2,2,2.0,0.0,0.0
2017-03-10,29235.325908,30039.128650,2,3,2.0,0.0,1.0
2017-03-20,32589.526731,29797.047476,3,2,3.0,0.0,1.0


In [70]:
total_calculated_error = result_df['calculated_err'].sum(axis = 0, skipna = True)
total_predicted_error = result_df['err'].sum(axis = 0, skipna = True)
print(total_calculated_error)
print(total_predicted_error)

4.0
41.0


In [71]:
result_df['calculated_err'].value_counts()

0.0    188
1.0      4
Name: calculated_err, dtype: int64

In [72]:
result_df['err'].value_counts()

0.0    151
1.0     41
Name: err, dtype: int64

In [73]:
# Stage 2 analysis
bad_predictions_df = result_df[result_df.err > 0.0]
false_positive = bad_predictions_df[bad_predictions_df.predicted_revenue>bad_predictions_df.revenue].shape[0]
print("predicted > true: ", false_positive)
print("predicted < true: ", bad_predictions_df.shape[0] - false_positive)

predicted > true:  21
predicted < true:  20


In [74]:
bad_predictions_df

,revenue,predicted_revenue,calculated_label,predicted_label,label,calculated_err,err
arrival_date,,,,,,,
2015-07-06,21441.591688,19320.385399,2,1,2.0,0.0,1.0
2015-07-20,36553.060216,40031.188600,3,4,3.0,0.0,1.0
2015-07-24,29268.000727,30949.814469,2,3,2.0,0.0,1.0
2015-07-25,38457.199769,46804.858410,3,4,3.0,0.0,1.0
2015-08-08,45576.442646,53781.842222,4,5,4.0,0.0,1.0
2015-08-10,62397.729899,56517.942947,6,5,6.0,0.0,1.0
2015-08-15,57467.688304,68567.309157,5,6,5.0,0.0,1.0
2015-08-27,17194.888688,20642.435167,1,2,1.0,0.0,1.0
2015-09-08,21819.951079,19077.198227,2,1,2.0,0.0,1.0


In [23]:
# Train with all training data
# Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=1126)

# Decide sample weight
sample_weights = raw_train_df.apply(lambda row: pow(row.expected_cost, 1), axis=1)

# Selecting parameters when Grid Searching
param_grid = {
    'transform__num__si__strategy': ['median'],
    'svd__n_components': [40, 45, 50, 55],
    'lr__penalty': ['l2'],
    'lr__C': [80, 90, 100]
}

gs = GridSearchCV(lr_pipe, param_grid, cv=kf, scoring=make_scorer(log_loss))
gs.fit(raw_train_df, raw_train_label, lr__sample_weight=sample_weights)

print(gs.best_params_)
print(gs.best_score_)

# Getting all the grid search results in a Pandas DataFrame
print(pd.DataFrame(gs.cv_results_))

{'lr__C': 100, 'lr__penalty': 'l2', 'svd__n_components': 50, 'transform__num__si__strategy': 'median'}
6.168234519787664
   mean_fit_time  std_fit_time  mean_score_time  std_score_time param_lr__C  \
0       6.608865      0.774375         0.116733        0.001261          80   
1       7.703421      0.955715         0.117696        0.000803          80   
2       7.749588      1.141011         0.118780        0.001637          80   
3       5.818701      0.870916         0.115441        0.002192          90   
4       7.891132      1.091097         0.117737        0.001571          90   
5       7.681377      0.321138         0.120124        0.002071          90   
6       6.906748      0.386835         0.117435        0.003943         100   
7       7.643118      0.796039         0.119949        0.002208         100   
8       7.793081      0.438887         0.119108        0.001209         100   

  param_lr__penalty param_svd__n_components  \
0                l2                      

In [102]:
# Make prediction on test data
test_df = pd.read_csv("test.csv")
test_nolabel_df = pd.read_csv("test_nolabel.csv")

In [103]:
test_df['arrival_date'] = test_df.apply(getArrivalDate, axis=1)
test_df['stays'] = test_df.apply(lambda row: row.stays_in_weekend_nights + row.stays_in_week_nights, axis=1)

In [104]:
test = test_df[featureCols]

In [105]:
# Use soft classification
will_be_canceled_prob = lr_pipe.predict_proba(test)
will_be_canceled = np.empty(shape=will_be_canceled_prob.shape[0])
for i in range(will_be_canceled.shape[0]):
    will_be_canceled[i] = will_be_canceled_prob[i][1]
will_be_canceled = pd.Series(will_be_canceled)

test_df['will_be_canceled'] = will_be_canceled
print(test_df['will_be_canceled'].value_counts())

0.991009    99
0.992079    84
0.986368    79
0.983138    69
0.990051    60
            ..
0.197696     1
0.373286     1
0.138229     1
0.000002     1
0.113375     1
Name: will_be_canceled, Length: 21546, dtype: int64


In [107]:
test_df

,ID,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests,arrival_date,stays,will_be_canceled
0,91531,City Hotel,75,2017,April,13,1,2,5,2,...,No Deposit,9.0,NaN,0,Transient,0,1,2017-04-01,7,0.233249
1,91532,City Hotel,208,2017,April,13,1,4,10,2,...,No Deposit,9.0,NaN,0,Transient,0,1,2017-04-01,14,0.615484
2,91533,Resort Hotel,12,2017,April,13,1,2,5,2,...,No Deposit,40.0,NaN,0,Contract,0,1,2017-04-01,7,0.091599
3,91534,City Hotel,76,2017,April,13,1,2,5,3,...,No Deposit,9.0,NaN,0,Transient,0,0,2017-04-01,7,0.627520
4,91535,City Hotel,9,2017,April,13,1,2,4,2,...,No Deposit,9.0,NaN,0,Transient,0,0,2017-04-01,6,0.497549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27854,119385,Resort Hotel,108,2017,August,35,31,2,5,2,...,No Deposit,241.0,NaN,0,Transient,0,1,2017-08-31,7,0.244351
27855,119386,Resort Hotel,194,2017,August,35,31,2,5,2,...,No Deposit,240.0,NaN,0,Transient,1,1,2017-08-31,7,0.000019
27856,119387,Resort Hotel,17,2017,August,35,31,0,3,2,...,No Deposit,240.0,NaN,0,Transient,0,2,2017-08-31,3,0.116379
27857,119388,Resort Hotel,191,2017,August,35,31,2,5,2,...,No Deposit,40.0,NaN,0,Contract,0,0,2017-08-31,7,0.075041


In [106]:
# Post process
test_df['predicted_revenue'] = test_df.apply(predictRevenue, axis=1)

AttributeError: 'Series' object has no attribute 'expected_cost'

In [99]:
# Aggregate by date
daily_revenue_df = test_df.groupby(['arrival_date']).agg({'predicted_revenue':'sum'})

In [100]:
daily_revenue_df

,predicted_revenue
arrival_date,
2017-04-01,15612.442258
2017-04-02,18659.357747
2017-04-03,39895.946151
2017-04-04,22021.079110
2017-04-05,31448.176362
...,...
2017-08-27,0.000000
2017-08-28,0.000000
2017-08-29,0.000000


In [ ]:
daily_revenue_df['predicted_label'] = pd.cut(daily_revenue_df.predicted_revenue, bins=thresholds,labels=ranks)

In [ ]:
test_nolabel_df = test_nolabel_df.set_index('arrival_date').join(daily_revenue_df, how='inner')